# NLP Assignment Group 036

Team :

1. GUMMALURI VENKATA VAIBHAV KARTIK - 2020FA04011
2. SUDHIR BABU BITRA - 2020SC04269
3. NANDA KUMAR - 2020SC04220

Import Necessary libraries

In [ ]:
import pandas as pd

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
from time import time
%matplotlib inline

In [3]:
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords

# Download the file and set it as a Dataframe

In [4]:
movieDf = pd.read_csv('movie_review.csv')

In [5]:
movieDf.head(10)

,fold_id,cv_tag,html_id,sent_id,text,tag
0,0,cv000,29590,0,films adapted from comic books have had plenty...,pos
1,0,cv000,29590,1,"for starters , it was created by alan moore ( ...",pos
2,0,cv000,29590,2,to say moore and campbell thoroughly researche...,pos
3,0,cv000,29590,3,"the book ( or "" graphic novel , "" if you will ...",pos
4,0,cv000,29590,4,"in other words , don't dismiss this film becau...",pos
5,0,cv000,29590,5,if you can get past the whole comic book thing...,pos
6,0,cv000,29590,6,getting the hughes brothers to direct this see...,pos
7,0,cv000,29590,7,"the ghetto in question is , of course , whitec...",pos
8,0,cv000,29590,8,"it's a filthy , sooty place where the whores (...",pos
9,0,cv000,29590,9,"when the first stiff turns up , copper peter g...",pos


In [6]:
movieDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64720 entries, 0 to 64719
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   fold_id  64720 non-null  int64 
 1   cv_tag   64720 non-null  object
 2   html_id  64720 non-null  int64 
 3   sent_id  64720 non-null  int64 
 4   text     64720 non-null  object
 5   tag      64720 non-null  object
dtypes: int64(3), object(3)
memory usage: 3.0+ MB


Remove un-used & unimportant columns

In [7]:
movieDf = movieDf.drop(['fold_id', 'html_id', 'sent_id','cv_tag'], axis=1)

In [8]:
movieDf.head(10)

,text,tag
0,films adapted from comic books have had plenty...,pos
1,"for starters , it was created by alan moore ( ...",pos
2,to say moore and campbell thoroughly researche...,pos
3,"the book ( or "" graphic novel , "" if you will ...",pos
4,"in other words , don't dismiss this film becau...",pos
5,if you can get past the whole comic book thing...,pos
6,getting the hughes brothers to direct this see...,pos
7,"the ghetto in question is , of course , whitec...",pos
8,"it's a filthy , sooty place where the whores (...",pos
9,"when the first stiff turns up , copper peter g...",pos


In [9]:
movieDf.columns = ['text', 'class']
movieDf.head(10)

,text,class
0,films adapted from comic books have had plenty...,pos
1,"for starters , it was created by alan moore ( ...",pos
2,to say moore and campbell thoroughly researche...,pos
3,"the book ( or "" graphic novel , "" if you will ...",pos
4,"in other words , don't dismiss this film becau...",pos
5,if you can get past the whole comic book thing...,pos
6,getting the hughes brothers to direct this see...,pos
7,"the ghetto in question is , of course , whitec...",pos
8,"it's a filthy , sooty place where the whores (...",pos
9,"when the first stiff turns up , copper peter g...",pos


In [10]:
count_class = pd.value_counts(movieDf["class"])
count_class

pos    32937
neg    31783
Name: class, dtype: int64

In [11]:
for i in range(10): 
    print(movieDf['text'][i])

films adapted from comic books have had plenty of success , whether they're about superheroes ( batman , superman , spawn ) , or geared toward kids ( casper ) or the arthouse crowd ( ghost world ) , but there's never really been a comic book like from hell before .
for starters , it was created by alan moore ( and eddie campbell ) , who brought the medium to a whole new level in the mid '80s with a 12-part series called the watchmen .
to say moore and campbell thoroughly researched the subject of jack the ripper would be like saying michael jackson is starting to look a little odd .
the book ( or " graphic novel , " if you will ) is over 500 pages long and includes nearly 30 more that consist of nothing but footnotes .
in other words , don't dismiss this film because of its source .
if you can get past the whole comic book thing , you might find another stumbling block in from hell's directors , albert and allen hughes .
getting the hughes brothers to direct this seems almost as ludicr

# Remove punctuations, special characters and stopwords from the text column. Convert the text to lower case.

All English Stop words

In [41]:
import nltk
nltk.download('stopwords')
stopwords_list = stopwords.words('english')
stopwords_list

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1123)>


['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [13]:
words_new = [i for i in movieDf['text'][4].split() if i.lower() not in stopwords.words('english')]
' '.join(words_new)

'words , dismiss film source .'

In [14]:
words_new

['words', ',', 'dismiss', 'film', 'source', '.']

Remove Puntuation and Special Characters

In [15]:
punc_string = string.punctuation
print(punc_string)
print(len(punc_string))

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
32


In [16]:
stemmer = SnowballStemmer('english')
print(' '.join([stemmer.stem(word) for word in movieDf['text'][1].split()]))

for starter , it was creat by alan moor ( and eddi campbel ) , who brought the medium to a whole new level in the mid 80s with a 12-part seri call the watchmen .


Put all pre-prcoessing steps as mentioned in the question in this method

In [17]:
def preprocess(text):
    # Remove stopwords
    words = [i for i in text.split() if i.lower() not in stopwords_list]
    text_1 = ' '.join(words)
    # Remove puctuation and replace with a space.
    text_2 = text_1.translate(str.maketrans(punc_string, len(punc_string)*' '))
    words_1 = text_2.split()
    # Perform word stemming 
    words_2 = [stemmer.stem(word) for word in words_1]
    text_3 = ' '.join(words_2)
    return text_3.lower()

Apply the pre-processing steps 

In [18]:
movieDf['text'] = movieDf['text'].apply(preprocess)

In [19]:
movieDf.head(10)

,text,class
0,film adapt comic book plenti success whether t...,pos
1,starter creat alan moor eddi campbel brought m...,pos
2,say moor campbel thorough research subject jac...,pos
3,book graphic novel 500 page long includ near 3...,pos
4,word dismiss film sourc,pos
5,get past whole comic book thing might find ano...,pos
6,get hugh brother direct seem almost ludicr cas...,pos
7,ghetto question cours whitechapel 1888 london ...,pos
8,filthi sooti place whore call unfortun start g...,pos
9,first stiff turn copper peter godley robbi col...,pos


# Create two objects X and y. X will be the 'text' column dataframe and y will be the “tag” column. create a CountVectorizer object and split the data into training and testing sets. Train a MultinomialNB model and Display the confusion Matrix

In [20]:
count_v = CountVectorizer()
word_count_matrix = count_v.fit_transform(movieDf['text'])
word_count_matrix

<64720x25367 sparse matrix of type '<class 'numpy.int64'>'
	with 698791 stored elements in Compressed Sparse Row format>

In [21]:
print(word_count_matrix)

  (0, 8028)	1
  (0, 620)	1
  (0, 4558)	2
  (0, 2761)	2
  (0, 17019)	1
  (0, 21626)	1
  (0, 24680)	1
  (0, 22396)	1
  (0, 18092)	1
  (0, 21738)	1
  (0, 2057)	1
  (0, 21744)	1
  (0, 20906)	1
  (0, 8875)	1
  (0, 22791)	1
  (0, 12187)	1
  (0, 3690)	1
  (0, 1449)	1
  (0, 5228)	1
  (0, 9006)	1
  (0, 24995)	1
  (0, 22374)	1
  (0, 15235)	1
  (0, 18120)	1
  (0, 12982)	1
  :	:
  (64716, 20398)	1
  (64716, 11038)	1
  (64716, 19005)	1
  (64717, 24478)	1
  (64717, 13271)	1
  (64717, 15793)	1
  (64717, 4546)	1
  (64717, 3904)	1
  (64717, 1717)	1
  (64717, 10044)	1
  (64717, 2675)	1
  (64717, 20398)	1
  (64717, 20633)	1
  (64717, 19005)	1
  (64718, 22340)	1
  (64718, 24935)	1
  (64718, 3245)	1
  (64718, 23759)	1
  (64719, 24478)	1
  (64719, 12767)	1
  (64719, 7497)	1
  (64719, 22342)	1
  (64719, 15332)	1
  (64719, 1558)	1
  (64719, 19005)	1


In [22]:
count_list = word_count_matrix.toarray().sum(axis=0)
word_list = count_v.get_feature_names()

In [23]:
word_freq = pd.DataFrame(count_list, index=word_list, columns=['Freq'])
word_freq.sort_values(by='Freq', ascending=False).head(30)

,Freq
film,11201
movi,6980
one,6031
like,4138
charact,3881
make,3243
get,3220
time,3047
scene,2671
even,2615


In [24]:
text_freq = pd.DataFrame(word_count_matrix.toarray()[23], index=count_v.get_feature_names(), columns=['word freq'])
# remove the rows with 0 frequency count
text_freq = text_freq[text_freq['word freq']!=0]
text_freq

,word freq
film,1
good,1
howev,1


Split into X & Y based on input and output

In [25]:
X = movieDf['text']
y = movieDf['class']

Split into test-train with 30 percent

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Create a pipeline with CountVectoiser and Multinomial NB

In [27]:
pipe = Pipeline([
    ('vector', CountVectorizer()),
    ('mulNB', MultinomialNB())
])

In [28]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('vector', CountVectorizer()), ('mulNB', MultinomialNB())])

In [29]:
y_pred = pipe.predict(X_test)

In [30]:
pipe.score(X_test, y_test)

0.705552121961269

In [31]:
from sklearn import metrics

The below is the confusion matrix

In [32]:
metrics.confusion_matrix(y_test, y_pred)

array([[6696, 2860],
       [2857, 7003]])

In [33]:
print(pipe.score(X_train, y_train))
print(pipe.score(X_test, y_test))

0.7854273353346283
0.705552121961269


# Display the POS tagging on the first 4 rows of ‘text’

In [35]:
movieDfPOSTaggingDf = movieDf.head(4)['text']

In [36]:
movieDfPOSTaggingDf.head(10)

0    film adapt comic book plenti success whether t...
1    starter creat alan moor eddi campbel brought m...
2    say moor campbel thorough research subject jac...
3    book graphic novel 500 page long includ near 3...
Name: text, dtype: object

In [37]:
movieDfPOSTaggingArray = movieDfPOSTaggingDf.to_numpy()

In [38]:
movieDfPOSTaggingArray

array(['film adapt comic book plenti success whether they re superhero batman superman spawn gear toward kid casper arthous crowd ghost world there s never realli comic book like hell',
       'starter creat alan moor eddi campbel brought medium whole new level mid 80s 12 part seri call watchmen',
       'say moor campbel thorough research subject jack ripper would like say michael jackson start look littl odd',
       'book graphic novel 500 page long includ near 30 consist noth footnot'],
      dtype=object)

In [39]:
import nltk

Below we have printed the first 4 texts and its Parsed Tags POS

In [40]:
for posTaggingElement in movieDfPOSTaggingArray:
    print("\n")
    print("Text is " + posTaggingElement + " ...")
    tokens = nltk.word_tokenize(posTaggingElement)
    tag = nltk.pos_tag(tokens)
    grammar = "NP: {<DT>?<JJ>*<NN>}"
    cp  =nltk.RegexpParser(grammar)
    result = cp.parse(tag)
    print(result)

    



Text is film adapt comic book plenti success whether they re superhero batman superman spawn gear toward kid casper arthous crowd ghost world there s never realli comic book like hell ...
(S
  (NP film/NN)
  adapt/NNS
  (NP comic/JJ book/NN)
  (NP plenti/NN)
  (NP success/NN)
  whether/IN
  they/PRP
  re/VBP
  (NP superhero/JJ batman/NN)
  (NP superman/NN)
  (NP spawn/NN)
  gear/VBP
  toward/IN
  kid/FW
  casper/FW
  arthous/JJ
  crowd/NNS
  ghost/VBD
  (NP world/NN)
  there/RB
  s/VBZ
  never/RB
  realli/VBN
  (NP comic/JJ book/NN)
  like/IN
  (NP hell/NN))


Text is starter creat alan moor eddi campbel brought medium whole new level mid 80s 12 part seri call watchmen ...
(S
  starter/RB
  (NP creat/NN)
  (NP alan/JJ moor/JJ eddi/NN)
  (NP campbel/NN)
  brought/VBD
  (NP medium/JJ whole/JJ new/JJ level/NN)
  mid/VBD
  80s/CD
  12/CD
  (NP part/NN)
  (NP seri/NN)
  (NP call/NN)
  watchmen/NNS)


Text is say moor campbel thorough research subject jack ripper would like say michael jac

# Build and display a dependency parser tree for the sentence:
“Following Jon's death in a mutiny, he is one of Jon's loyalists who find his body and barricade themselves inside his quarters, refusing to acknowledge Thorne's leadership.”

In [2]:
!pip install spacy && python -m spacy download en

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 27.1 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


Below is the dependency Parser Tree for the mentioned sentence

In [4]:
import spacy
from nltk import Tree


en_nlp = spacy.load('en_core_web_sm')

doc = en_nlp("Following Jon's death in a mutiny, he is one of Jon's loyalists who find his body and barricade themselves inside his quarters, refusing to acknowledge Thorne's leadership")

def to_nltk_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(node.orth_, [to_nltk_tree(child) for child in node.children])
    else:
        return node.orth_


[to_nltk_tree(sent.root).pretty_print() for sent in doc.sents]

         is                                                                                            
  _______|_______________________________                                                               
 |   |          |                       one                                                            
 |   |          |                        |                                                              
 |   |          |                        of                                                            
 |   |          |                        |                                                              
 |   |          |                    loyalists                                                         
 |   |          |              __________|______________                                                
 |   |          |             |                        find                                            
 |   |          |             |    _____________________|___

[None]